In [28]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import LocallyLinearEmbedding
from xgboost import XGBClassifier, plot_importance
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

patient_id = 2

val_csv_path = "D:\\Faculdade\\TCC\\dados\\epilepsy_ecosystem\\extracted_features_val.csv"

df_val = pd.read_csv(val_csv_path)

if patient_id:
    train_csv_path = "D:\\Faculdade\\TCC\\dados\\epilepsy_ecosystem\\selected_features_Pat{}Train.csv".format(patient_id)
    df_val = df_val.loc[df_val['patient'] == patient_id]
else:
    train_csv_path = "D:\\Faculdade\\TCC\\dados\\epilepsy_ecosystem\\selected_features.csv"

df_train = pd.read_csv(train_csv_path)
df_train = df_train[df_train.columns[1:]]

df_val = df_val[df_train.columns] # ignoring index column

print(df_train, df_val)
df_train = df_train.drop(columns=['patient','segment_id'])


X_train = df_train.drop(columns=['class'])
y_train = df_train['class']


       ch_0_delta2_skewness  ch_0_delta2_kurtosis  ch_0_abs_mean  \
0                 -2.317723             47.804662       0.830204   
1                 -2.485734             43.172208       0.465991   
2                 -2.739372             50.373893       0.215549   
3                 -3.353047             54.377538       4.016088   
4                 -2.949368             53.449676       0.125409   
...                     ...                   ...            ...   
33865             -2.257567             43.239930       4.464105   
33866             -2.817637             43.885165       8.100379   
33867             -2.684656             51.492346       9.442240   
33868             -2.964794             54.071197       6.971988   
33869             -1.861420             39.258907       6.378707   

       ch_0_complexity  ch_0_delta  ch_0_theta  ch_0_alpha  ch_0_beta  \
0             5.193336   -1.677645   -2.113039   -2.379060  -2.405168   
1             5.449044   -1.558841   

In [29]:
print(X_train)
print(y_train)

print(df_val)

       ch_0_delta2_skewness  ch_0_delta2_kurtosis  ch_0_abs_mean  \
0                 -2.317723             47.804662       0.830204   
1                 -2.485734             43.172208       0.465991   
2                 -2.739372             50.373893       0.215549   
3                 -3.353047             54.377538       4.016088   
4                 -2.949368             53.449676       0.125409   
...                     ...                   ...            ...   
33865             -2.257567             43.239930       4.464105   
33866             -2.817637             43.885165       8.100379   
33867             -2.684656             51.492346       9.442240   
33868             -2.964794             54.071197       6.971988   
33869             -1.861420             39.258907       6.378707   

       ch_0_complexity  ch_0_delta  ch_0_theta  ch_0_alpha  ch_0_beta  \
0             5.193336   -1.677645   -2.113039   -2.379060  -2.405168   
1             5.449044   -1.558841   

# Applying PCA in order to remove potentially correlated features

In [30]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from joblib import dump

apply_PCA = False
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

if apply_PCA:
    pca = PCA(n_components = 0.99)
    X_train_final = pca.fit_transform(X_train_scaled)
    print("Remaining features : {}".format(X_train_final.shape[1]))
    print("Explained variance ratio : {}".format(sum(pca.explained_variance_ratio_)))
else:
    pca = None
    X_train_final = X_train_scaled
    

<h1> Training - Random Forest and ExtraTrees </h1>

In [31]:
# training
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

params = dict(
            n_estimators=150,
            class_weight='balanced',
            max_depth=10,
            bootstrap=True,
            oob_score=True,
            n_jobs=-1
        )

rdf = RandomForestClassifier(**params)
xt_trees = ExtraTreesClassifier(**params)

print('Fitting...')
rdf.fit(X_train_final, y_train)
xt_trees.fit(X_train_final, y_train)

print("OOB Score - Random Forest: {}".format(rdf.oob_score_))
print("OOB Score - Extra Trees: {}".format(xt_trees.oob_score_))






Fitting...
OOB Score - Random Forest: 0.7843814585178625
OOB Score - Extra Trees: 0.6920578683200472


In [32]:
from evaluation import evaluate_from_df

evaluate_from_df(rdf, scaler, pca, df_val, agg_method='max_proba')
evaluate_from_df(xt_trees, scaler, pca, df_val, agg_method='max_proba')

evaluate_from_df(rdf, scaler, pca, df_val, agg_method='avg_proba')
evaluate_from_df(xt_trees, scaler, pca, df_val, agg_method='avg_proba')

[[238  22]
 [ 12  21]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       260
           1       0.49      0.64      0.55        33

    accuracy                           0.88       293
   macro avg       0.72      0.78      0.74       293
weighted avg       0.90      0.88      0.89       293

ROC AUC Score : 0.7758741258741259
[[209  51]
 [ 10  23]]
              precision    recall  f1-score   support

           0       0.95      0.80      0.87       260
           1       0.31      0.70      0.43        33

    accuracy                           0.79       293
   macro avg       0.63      0.75      0.65       293
weighted avg       0.88      0.79      0.82       293

ROC AUC Score : 0.7504079254079254
[[221  39]
 [  8  25]]
              precision    recall  f1-score   support

           0       0.97      0.85      0.90       260
           1       0.39      0.76      0.52        33

    accuracy                           0.84

# XGBoost

In [33]:
params = dict(
        class_weight='balanced',
        max_depth=10,
        subsample=0.5,
        n_jobs=-1
    )


xgb = XGBClassifier(**params)
xgb.fit(X_train_final, y_train)



XGBClassifier(base_score=0.5, booster='gbtree', class_weight='balanced',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [34]:
evaluate_from_df(xgb, scaler, pca, df_val, agg_method='max_proba')
evaluate_from_df(xgb, scaler, pca, df_val, agg_method='avg_proba')

[[259   1]
 [ 30   3]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.94       260
           1       0.75      0.09      0.16        33

    accuracy                           0.89       293
   macro avg       0.82      0.54      0.55       293
weighted avg       0.88      0.89      0.86       293

ROC AUC Score : 0.5435314685314685
[[259   1]
 [ 29   4]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       260
           1       0.80      0.12      0.21        33

    accuracy                           0.90       293
   macro avg       0.85      0.56      0.58       293
weighted avg       0.89      0.90      0.86       293

ROC AUC Score : 0.5586829836829836


# Logistic Regression



In [35]:
lr_clf = LogisticRegression(class_weight='balanced')
lr_clf2 = LogisticRegression(class_weight={0: 0.2, 1:0.8})
lr_clf3 = LogisticRegression(class_weight={0: 0.1, 1:0.9})

lr_clf.fit(X_train_final, y_train)
lr_clf2.fit(X_train_final, y_train)
lr_clf3.fit(X_train_final, y_train)

C:\Users\Leonardo\Anaconda2\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Leonardo\Anaconda2\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Leonardo\Anaconda2\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight={0: 0.1, 1: 0.9}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
from evaluation import evaluate_from_df

evaluate_from_df(lr_clf, scaler, pca, df_val, agg_method='max_proba')
evaluate_from_df(lr_clf2, scaler, pca, df_val, agg_method='max_proba')
evaluate_from_df(lr_clf3, scaler, pca, df_val, agg_method='max_proba')

evaluate_from_df(lr_clf, scaler, pca, df_val, agg_method='avg_proba')
evaluate_from_df(lr_clf2, scaler, pca, df_val, agg_method='avg_proba')
evaluate_from_df(lr_clf3, scaler, pca, df_val, agg_method='avg_proba')

[[205  55]
 [  5  28]]
              precision    recall  f1-score   support

           0       0.98      0.79      0.87       260
           1       0.34      0.85      0.48        33

    accuracy                           0.80       293
   macro avg       0.66      0.82      0.68       293
weighted avg       0.90      0.80      0.83       293

ROC AUC Score : 0.8184731934731935
[[238  22]
 [ 14  19]]
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       260
           1       0.46      0.58      0.51        33

    accuracy                           0.88       293
   macro avg       0.70      0.75      0.72       293
weighted avg       0.89      0.88      0.88       293

ROC AUC Score : 0.7455710955710956
[[184  76]
 [  2  31]]
              precision    recall  f1-score   support

           0       0.99      0.71      0.83       260
           1       0.29      0.94      0.44        33

    accuracy                           0.73